In [ ]:
# Pydantic/JSON 파서가 더 강력하지만, 로컬 모델은 잘 안되는 경우도 있어서 대안으로 사용
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from dotenv import load_dotenv
import torch
import sys
import os
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, AutoProcessor, AutoModelForVision2Seq
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_core.runnables import RunnableLambda
from PIL import Image

os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
load_dotenv()

True

In [2]:
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(name="source", description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [3]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.
}
```


In [4]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions":format_instructions},
)

In [11]:
# 허깅페이스 모델 업로드 (8bit 양자화)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", trust_remote_code=True)
quantization_config = BitsAndBytesConfig(load_in_8bit=True,llm_int8_threshold=6.0)          # 8bit 양자화 설정
model = AutoModelForVision2Seq.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    quantization_config=quantization_config,  # 8bit 양자화
    device_map="cuda:0",
    trust_remote_code=True,
)
pipe = pipeline(
    "image-text-to-text",
    model=model,
    processor=processor,
    max_new_tokens=40,
    temperature=0.1,
    do_sample=True,    # 샘플링 활성화
    # return_full_text=False,
    # pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 명시적 설정
)
llm = HuggingFacePipeline(pipeline=pipe)

c:\Python312\Lib\site-packages\transformers\models\auto\modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4204\4100203956.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [12]:
answer = llm.invoke("안녕, 너 한국어 좀 치니?")
print(answer)

ValueError: Got invalid task image-text-to-text, currently only ('text2text-generation', 'text-generation', 'summarization', 'translation') are supported

In [ ]:
# 4비트 양자화
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

bnb_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # fp16도 가능
)

model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    quantization_config=bnb_4bit,
    device_map={"": 0},
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # fp16도 가능
)

pipe = pipeline(
    "image-text-to-text",
    model=model,
    processor=processor,
    max_new_tokens=64,
    temperature=0.1,
    do_sample=True,
)

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
c:\Python312\Lib\site-packages\transformers\models\auto\modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


In [13]:
# llm = HuggingFacePipeline(pipeline=pipe)

import asyncio
vl = RunnableLambda(lambda x: pipe(x)[0]["generated_text"])
img = Image.open(r"C:\Users\Administrator\Downloads\Gemini Board.png").convert("RGB")
answer = await vl.ainvoke({"image": img, "text": "이미지 설명해줘"})
print(answer)

KeyError: 'images'